In [2]:
from peft import LoraConfig
from trl import DPOTrainer
from transformers import TrainingArguments

/home/vulcan/.local/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

torch.cuda.is_available()

True

In [4]:
import json
from datasets import Dataset

with open("unique_healthcare_preference_data_100.json", "r") as f:
    preference_data = json.load(f)

dataset = Dataset.from_list(preference_data)
dataset


Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 100
})

In [5]:
preference_data

[{'prompt': 'What should I do if I accidentally took two doses of my blood pressure medication?',
  'chosen': "It's important to seek medical advice immediately. Double dosing can lower your blood pressure excessively. Please call your doctor or go to the nearest emergency room for evaluation.",
  'rejected': "You'll be fine. Just don't do it again."},
 {'prompt': 'I’m having trouble sleeping for the past week. Any suggestions?',
  'chosen': "I'm sorry to hear you're struggling with sleep. You might consider maintaining a consistent bedtime routine and avoiding screens before bed. Please speak with a healthcare provider if the issue persists.",
  'rejected': 'Everyone has sleepless nights. Just relax and watch something boring.'},
 {'prompt': 'Are headaches after COVID-19 normal?',
  'chosen': "Headaches can occur after recovering from COVID-19, but it's important to monitor your symptoms. If they persist or worsen, please consult a healthcare provider for evaluation.",
  'rejected': '

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from trl import DPOTrainer
from peft import LoraConfig
import torch

# Model and tokenizer initialization
# model_id = "google/medgemma-4b-it"
model_id = 'stanford-crfm/BioMedLM'

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16 if torch.cuda.is_bf16_supported() else torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)

tokenizer.pad_token = tokenizer.eos_token

from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")


# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    bias="none",
    task_type="CAUSAL_LM",
    lora_dropout=0.05,
    target_modules=["attn.c_attn", "attn.c_proj", "mlp.c_fc", "mlp.c_proj"]    
    # target_modules=["q_proj", "v_proj", "k_proj", "o_proj"]
)

# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-5,
    lr_scheduler_type="cosine",
    num_train_epochs=5,
    logging_steps=10,
    output_dir="/home/vulcan/Documents/Projects/Fine Tuning /dpo_results",
    optim="paged_adamw_8bit",
    report_to="none",
    fp16=True,
)

# Initialize DPOTrainer
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # Explicitly set to None for LoRA-based DPO
    args=training_args,
    train_dataset=dataset,  # Ensure 'dataset' is defined
    peft_config=peft_config,
    beta=0.1,  # DPO-specific argument
    tokenizer=tokenizer,
)

# Start training
print("Starting DPO training...")

dpo_trainer.train()
print("DPO training complete!")

# Save the fine-tuned model
dpo_trainer.save_model("health_aligned_dpo")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Starting DPO training...


  0%|          | 0/75 [00:00<?, ?it/s]

Could not estimate the number of tokens of the input, floating-point operations will not be computed


{'loss': 0.6478, 'learning_rate': 8.666666666666668e-06, 'rewards/chosen': 0.05083867907524109, 'rewards/rejected': -0.04363936930894852, 'rewards/accuracies': 0.8999999761581421, 'rewards/margins': 0.09447804093360901, 'logps/rejected': -66.34424591064453, 'logps/chosen': -79.46990203857422, 'logits/rejected': -3.6819376945495605, 'logits/chosen': -4.664607048034668, 'epoch': 0.4}
{'loss': 0.5544, 'learning_rate': 7.333333333333333e-06, 'rewards/chosen': 0.16416513919830322, 'rewards/rejected': -0.14142093062400818, 'rewards/accuracies': 1.0, 'rewards/margins': 0.3055860996246338, 'logps/rejected': -67.09962463378906, 'logps/chosen': -81.68949890136719, 'logits/rejected': -3.453129291534424, 'logits/chosen': -4.296846866607666, 'epoch': 0.8}
{'loss': 0.4432, 'learning_rate': 6e-06, 'rewards/chosen': 0.32381194829940796, 'rewards/rejected': -0.27181780338287354, 'rewards/accuracies': 1.0, 'rewards/margins': 0.5956298112869263, 'logps/rejected': -67.67825317382812, 'logps/chosen': -79.0